In [1]:
from utilities import *
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
############### User Configuration ###############
##################################################

datadir = "." # All ZDA files in this directory + subdirectories are loaded
selected_filename = "05_01_05" # focus for subsequent analysis

In [5]:
############## Driver script: begin ##############
##################################################

# Load data
data_loader = DataLoader()
data_loader.load_all_zda(data_dir=datadir + "/zda_targets")

# Select data of interest        
selected_data = data_loader.select_data_by_keyword(selected_filename)

# Clip to the time range 20 - 90 ms (get rid of the camera "foot", look at prestim only)
selected_data.clip_data(t_range=[40,180])

raw_data, meta, rli = selected_data.get_data(trial=0), selected_data.get_meta(), selected_data.get_rli()

# Need to subtract off the low-frequency voltage drift. First-order correction
tr = Tracer()
tr.correct_background(meta, raw_data)

#full trace
tr.plot_trace(raw_data[:,:,:], 
           40, 
           40, 
           meta['interval_between_samples'])

./zda_targets/05_01_05.zda
Number of files loaded: 1


In [8]:
# Frequency Decompositions

freq_analzyer = FreqAnalyzer()

x_fft = freq_analzyer.compute_fft_binning(meta)
y_fft_avg = None
n = raw_data.shape[0] * raw_data.shape[1]

for x in range(raw_data.shape[0]):
    for y in range(raw_data.shape[1]):
            
            y_fft = freq_analzyer.decompose_trace_frequencies(meta, 
                                                raw_data[x,y,:],
                                                x_fft=x_fft,
                                                lower_freq=0,
                                                upper_freq=50,
                                                y_max=2000,
                                                plot=False)
            if y_fft_avg is None:
                y_fft_avg = y_fft / n
            else:
                y_fft_avg += y_fft / n

In [9]:
# Mean frequency decomposition (averaged by frequency over all pixels)
plt.plot(x_fft, 
         np.abs(y_fft_avg))
plt.xlim([10, 400])
plt.ylim([500,600])
plt.title("Average Frequency Decomposition: All Pixels")
plt.show()
plt.savefig(selected_filename + "_avg_freq.jpg")

In [ ]:
# Average only for high-SNR and/or ROIs
# .... TO DO
asnr = AnalyzerSNR(trial)

snr = asnr.get_snr(plot=True)

print("max SNR:", np.max(snr), "min SNR:", np.min(snr))
asnr.cluster_on_snr(plot=True)